<a href="https://colab.research.google.com/github/cmesajil/grafica/blob/main/encubrimiento_y_matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# prompt: instala los paquetes jump y glpk usando pkg con julia

using Pkg
Pkg.add("JuMP")
Pkg.add("GLPK")

   Resolving package versions...
   Installed CodecBzip2 ───────── v0.8.5
   Installed NaNMath ──────────── v1.1.3
   Installed XZ_jll ───────────── v5.8.0+0
   Installed Fontconfig_jll ───── v2.16.0+0
   Installed BenchmarkTools ───── v1.6.0
   Installed MutableArithmetics ─ v1.6.4
   Installed XSLT_jll ─────────── v1.1.43+0
   Installed StructTypes ──────── v1.11.0
   Installed JSON3 ────────────── v1.14.2
   Installed Xorg_libICE_jll ──── v1.1.2+0
   Installed FFTW_jll ─────────── v3.3.11+0
   Installed IntervalArithmetic ─ v0.22.28
   Installed JuMP ─────────────── v1.25.0
   Installed MathOptInterface ─── v1.38.0
   Installed Graphite2_jll ────── v1.3.15+0
   Installed Xorg_xtrans_jll ──── v1.6.0+0
   Installed Libgcrypt_jll ────── v1.11.1+0
   Installed StructArrays ─────── v0.7.1
   Installed ForwardDiff ──────── v0.10.38
    Updating `~/.julia/environments/v1.10/Project.toml`
  [4076af6c] + JuMP v1.25.0
    Updating `~/.julia/environments/v1.10/Manifest.toml`
  [6e4b80f9] + Ben

In [22]:
# prompt: dame la adyacencia de un grafo que tenga de enlaces 1 2,1 4,2 3,2 10,3 2 ,3 4,3 7,3 6,3 5,4 1,4 10-, 4 3, 4 7 ,4 6,4 5 ,5 3, 5 9, 5 6 , 5 4 ,5 8 , 6 4 ,6 7 ,6 3, 6 9 ,6 5, 6 8, 7 4, 7 10 ,7 3 , 7 6 ,8 6 , 8 5 ,9 5 ,9 6 , 10 2, 10 4 ,10 7 . en codigo julia

# Número de nodos en el grafo
nodos = 10

# Inicializamos la matriz de adyacencia
adyacencia = zeros(Int, nodos, nodos)

# Lista de enlaces proporcionados
enlaces = [
    (1, 2), (1, 4),(2,1),(2, 3), (2, 10), (3, 2), (3, 4), (3, 7), (3, 6), (3, 5),
    (4, 1), (4, 10), (4, 3), (4, 7), (4, 6), (4, 5), (5, 3), (5, 9), (5, 6),
    (5, 4), (5, 8), (6, 4), (6, 7), (6, 3), (6, 9), (6, 5), (6, 8), (7, 4),
    (7, 10), (7, 3), (7, 6), (8, 6), (8, 5), (9, 5), (9, 6), (10, 2), (10, 4),
    (10, 7)
]

# Llenamos la matriz de adyacencia con 1 donde haya un enlace entre dos nodos
for (i, j) in enlaces
    adyacencia[i, j] = 1
    adyacencia[j, i] = 1  # Asumimos que el grafo es no dirigido
end

# Mostrar la matriz de adyacencia
println("Matriz de adyacencia:")
println(adyacencia)


Matriz de adyacencia:
[0 1 0 1 0 0 0 0 0 0; 1 0 1 0 0 0 0 0 0 1; 0 1 0 1 1 1 1 0 0 0; 1 0 1 0 1 1 1 0 0 1; 0 0 1 1 0 1 0 1 1 0; 0 0 1 1 1 0 1 1 1 0; 0 0 1 1 0 1 0 0 0 1; 0 0 0 0 1 1 0 0 0 0; 0 0 0 0 1 1 0 0 0 0; 0 1 0 1 0 0 1 0 0 0]


In [28]:
using JuMP
using GLPK



n= size(adyacencia, 2)
#Modelo de optimización
modelo = Model(GLPK.Optimizer)

# Variables de decisión (binarias)
@variable(modelo, x[1:n, 1:n], Bin)

# Función objetivo (minimizar el costo total)
@objective(modelo, Max, sum(adyacencia[i, j] * x[i, j] for i in 1:n, j in 1:n))

# Restricciones
# Cada trabajo debe ser asignado a una sola máquina
@constraint(modelo, [i in 1:n], sum(x[i, j] for j in 1:n) <= 1)

# Cada máquina debe ser asignada a un solo trabajo
@constraint(modelo, [j in 1:n], sum(x[i, j] for i in 1:n) <= 1)

# Resolver el modelo
optimize!(modelo)

# Mostrar la solución
println("cardinal del matching: ", objective_value(modelo))
println("x:")
for i in 1:n
    for j in 1:n
        if value(x[i,j]) ≈ 1
            println("x[$i,$j] ≈ 1")
        end
    end
end

LoadError: ParseError:
[90m# Error @ [0;0m]8;;file:///content/In[28]#21:44\[90mIn[28]:21:44[0;0m]8;;\
# Cada máquina debe ser asignada a un solo trabajo
@constraint(modelo, [j in 1:n], sum(x[i, j][48;2;120;70;70m[0;0mfor i in 1:n) <= 1)
[90m#                                          └ ── [0;0m[91mExpected space before `for` in generator[0;0m

In [24]:
adyacencia

10×10 Matrix{Int64}:
 0  1  0  1  0  0  0  0  0  0
 1  0  1  0  0  0  0  0  0  1
 0  1  0  1  1  1  1  0  0  0
 1  0  1  0  1  1  1  0  0  1
 0  0  1  1  0  1  0  1  1  0
 0  0  1  1  1  0  1  1  1  0
 0  0  1  1  0  1  0  0  0  1
 0  0  0  0  1  1  0  0  0  0
 0  0  0  0  1  1  0  0  0  0
 0  1  0  1  0  0  1  0  0  0

In [30]:
n= size(adyacencia, 2)
#Modelo de optimización
modelo2 = Model(GLPK.Optimizer)

# Variables de decisión (binarias)
@variable(modelo2, x[1:n], Bin)

# Función objetivo (minimizar el costo total)
@objective(modelo2, Max, sum(x[i] for i in 1:n))

# Restricciones
@constraint(modelo2,
    [i in 1:n, j in 1:n],
    sum(adyacencia[i, k] for k in 1:n) <= x[i] + x[j]
)

# Resolver el modelo
optimize!(modelo2)

# Mostrar la solución
println("cardinal del matching: ", objective_value(modelo2))
println("x:")
for i in 1:n
  println(value(x[i,j]))
end

LoadError: Result index of attribute MathOptInterface.ObjectiveValue(1) out of bounds. There are currently 0 solution(s) in the model.

In [ ]:
function resolver_reinas(n)
    modelo = Model(GLPK.Optimizer)  # Usamos el solver GLPK

    # Variables binarias x[i, j] = 1 si hay una reina en la celda (i, j), 0 en caso contrario
    @variable(modelo, x[1:n, 1:n], Bin)

    # Restricción 1: Una reina por fila
    for i in 1:n
        @constraint(modelo, sum(x[i, j] for j in 1:n) == 1)
    end

    # Restricción 2: Una reina por columna
    for j in 1:n
        @constraint(modelo, sum(x[i, j] for i in 1:n) == 1)
    end

    # Restricción 3: Diagonales principales (↘)
    for k in 2:(2n-1)
        @constraint(modelo, sum(x[i, j] for i in 1:n for j in 1:n if i + j == k) ≤ 1)
    end

    # Restricción 4: Diagonales secundarias (↙)
    for k in -n+1:n-1
        @constraint(modelo, sum(x[i, j] for i in 1:n for j in 1:n if i - j == k) ≤ 1)
    end

    # No necesitamos función objetivo (es un problema de factibilidad)

    # Resolver el modelo
    optimize!(modelo)

    # Verificar si hay solución óptima
    if termination_status(modelo) == MOI.OPTIMAL
        solucion = value.(x)  # Obtener los valores óptimos de las variables
        println("Solución encontrada para ", n, " reinas:")
        for i in 1:n
            println(join([solucion[i, j] == 1 ? "♛" : "." for j in 1:n], " "))
        end
    else
        println("No se encontró una solución óptima.")
    end
end

# Llamamos a la función para resolver el problema con un tablero de 10x10
resolver_reinas(10)